# Data and Setting

In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np
import random
import networkx as nx
from networkx.algorithms import bipartite
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

### Network

In [2]:
#dataframe for the network 

#names of columns
cols = ['edge_ID','source','target']

#loading Dataframe
df_edges = pd.read_csv('/Users/paolaantonicoli/Downloads/wikigraph_reduced.csv', delimiter = '\t',names = cols).drop(index = 0)

#making edge_ID as int
df_edges['edge_ID'] = list(map(int, df_edges['edge_ID']))

df_edges


,edge_ID,source,target
1,796,95,1185516
2,909,108,1059989
3,910,108,1062426
4,911,108,1161925
5,1141,134,541222
...,...,...,...
483090,28511768,1791481,1060990
483091,28511769,1791481,1061382
483092,28511771,1791481,1110380
483093,28511772,1791482,80740


In [119]:
#tryng to calculate in-degree from the DataFrame
df_edges.groupby('target').count().mean()

edge_ID    6.401394
source     6.401394
dtype: float64

### Page Names

In [3]:
df_names = pd.read_csv('/Users/paolaantonicoli/Downloads/wiki-topcats-page-names.txt',delimiter = '\t',names =['node name'])

In [4]:
df_names['node'] = [s.split()[0] for s in list(df_names['node name']) ]
df_names['name'] = [' '.join(s.split()[1:]) for s in list(df_names['node name'])]
df_names = df_names.drop('node name',axis = 1)

In [5]:
pages = df_names['node']

In [6]:
df_names

,node,name
0,0,Chiasmal syndrome
1,1,Kleroterion
2,2,Pinakion
3,3,LyndonHochschildSerre spectral sequence
4,4,Zariski's main theorem
...,...,...
1791484,1791484,Noadiah Russell (Yale founder)
1791485,1791485,Cornus officinalis
1791486,1791486,Peter Grummitt
1791487,1791487,Baron Estcourt


### Categories

In [7]:
names = ['cat_name','pages']
df_cat = pd.read_csv('/Users/paolaantonicoli/Downloads/wiki-topcats-categories.txt.gz',sep = ';',names = names)

df_cat['cat_name'] = df_cat.apply(lambda x: x['cat_name'][9:],axis = 1)

df_cat['pages'] = df_cat.apply(lambda x: x['pages'].split(),axis = 1)

df_cat

,cat_name,pages
0,Buprestoidea,"[301, 302, 303, 304, 305, 306, 307, 308, 309, ..."
1,People_from_Worcester,"[1056, 1057, 1058, 1059, 1060, 60971, 76515, 7..."
2,Skin_conditions_resulting_from_physical_factors,"[971, 973, 1166, 1167, 1168, 1169, 1170, 1171,..."
3,Visual_kei_bands,"[1297, 1300, 1311, 1312, 1313, 1314, 1315, 131..."
4,Japanese_rock_music_groups,"[1297, 1300, 1313, 1314, 1315, 1316, 1319, 132..."
...,...,...
17359,British_science_fiction_novels,"[16110, 24545, 39814, 67923, 108629, 108630, 1..."
17360,Television_soundtracks,"[1117, 110192, 110376, 110377, 110380, 110395,..."
17361,Diptera_of_North_America,"[279, 280, 283, 31110, 31129, 31133, 31138, 31..."
17362,Diptera_of_Asia,"[279, 31129, 31133, 31134, 31135, 31136, 31137..."


In [8]:
len(df_cat['cat_name'])

17364

In [10]:
#in order to remove the pages belonging to multiple categories, I made an inverted index key: Page, values: Categories in which the pages appears.
#than I randomly sampled the values in uniform way and built the dataframe again

categories = list(df_cat['cat_name'])
pages_for_cat =list(df_cat['pages'])


d_page_cat = defaultdict(list)
for i,cat in enumerate(categories):
    for pag in pages_for_cat[i]:
        d_page_cat[pag].append(cat)
    



In [11]:
list(d_page_cat.keys())[:10]

['301', '302', '303', '304', '305', '306', '307', '308', '309', '310']

In [12]:
d_random_cat_page = defaultdict(list)

for page in d_page_cat.keys():
    cat_of_page = random.sample(set(d_page_cat[page]),1)
    d_random_cat_page[cat_of_page[0]].append(page)

In [13]:
#some categories has been removed, since there is no pages in it or all the pages are other categories
new_df_cat = pd.DataFrame()
new_df_cat['cat_name'] = d_random_cat_page.keys()
new_df_cat['pages'] = d_random_cat_page.values()
new_df_cat

,cat_name,pages
0,Buprestoidea,"[301, 302, 303, 304, 305, 306, 307, 308, 309, ..."
1,Year_of_birth_missing,"[1056, 1535870, 289081, 1511631, 709705, 15368..."
2,"Alumni_of_New_College,_Oxford","[1057, 34050, 1390967, 48131, 66938, 449636, 1..."
3,Alumni_of_Loughborough_University,"[1058, 76518, 558126, 479434, 187362, 436721, ..."
4,People_from_Worcester,"[1059, 1060, 84108, 84740, 158412, 216650, 348..."
...,...,...
17355,Populated_places_in_Navarre,"[444707, 1781142, 1781146, 1781147, 1781153, 1..."
17356,Municipalities_of_South_Tyrol,"[215159, 443945, 943609, 1781498, 1781519, 178..."
17357,Suburbs_of_Canberra,"[1781950, 1781953, 1781955, 1781959, 1781960, ..."
17358,Redirects_from_UK_postcode_districts,"[1640637, 1640638, 1788784, 1788785]"


In [14]:
s1 = set(new_df_cat['cat_name'])
s2 = set(df_cat['cat_name'])
s2.difference(s1)

{'Colostethus',
 'Days_in_2004',
 'Disused_railway_stations_in_Pomeranian_Voivodeship',
 'Shanghai_Metro_stations'}

In [15]:
len(d_page_cat.values())

1791489

# Create Graph

# RQ 1

In [117]:
G=nx.convert_matrix.from_pandas_edgelist(df_edges, 'source', 'target', ['edge_ID'],create_using=nx.DiGraph())


In [118]:
nx.is_directed(G)



True

### 1. Is the graph directed?

### How many articles are we considering

In [104]:
len(G.nodes)




98343

### How many Hyperlinks 

In [99]:
len(G.edges)


483094

### Avarage Degree

In [100]:
dict_in_degree = dict(G.in_degree())
list_degree_in = list(dict_in_degree.values())
avg_degree_in = np.mean(np.array(list_degree_in))
avg_degree_in



4.912337431235573

In [101]:
list_degree_out


[1, 0, 1]

In [102]:
dict_out_degree = dict(G.out_degree())
list_degree_out = list(dict_out_degree.values())
avg_degree_out = np.mean(np.array(list_degree_out))
avg_degree_out

4.912337431235573